In [1]:
from random import randint
import time
import serial

In [2]:

def generate_packets(n, payload_length):
    def generate_payload(payload_length):
        if not payload_length == -1:
            return "".join([str(randint(0, 9)) for _ in range(payload_length)])
        else:
            return "".join([str(randint(0, 9)) for _ in range(randint(1, 256))])
    
    time_of_generation = int(time.time())

    return ["SOT", f"TIME={time_of_generation}"] + [generate_payload(payload_length) for _ in range(n)] + ["EOT"], time_of_generation

In [6]:
packets, _time = generate_packets(10, -1)
packets

['SOT',
 'TIME=1688028234',
 '3353736248808944',
 '46510690803966608696044869994812500390746954966537982331673423393625044904329696449477156414129190963371524798002601953107852495303952934289305510363794',
 '9428537877105425149525219097730533855387675160595044521041718294132564405985353791482',
 '680605692249771593417265660000806681606925368300404828669536772608992032389246857133066209572509230041654558793181818856058031424914929385',
 '01896444822640903767877121206561945925720562968356048103452479129',
 '84044962725531347807954490723481153196469724668635863925916180112907349368994406546587730347340267449846341388262590673879377731342758726004733',
 '78658915997523534067655207158417635540756803745652579272794734583229792803635829589613930161616903991685995641808268460718771061035391983338664709392809269183031269147268954253600181112204876408642021711813896824334122357869598458902685418',
 '46',
 '471575874845278528977188770899766675935227145677634833443225237264407924126043248606516375

['88292336556108042423',
 '04841572614870643747',
 '85414612702080514651',
 '13812308276386273097',
 '37796786377626528286',
 '05145011903638194450',
 '83232632020613866791',
 '54072909255583982317',
 '29322947387331296658',
 '77281424710304264148',
 'N=10',
 'TIME=1687865300',
 'SOT']

In [3]:
device = serial.Serial("COM8", 115200)

In [6]:
def AT(device, command: str, timeout=10):
    start_time = time.time()
    device.write((command + "\n").encode())
    response = ""
    
    while "OK" not in response:
        response = device.read_all().decode()
        if "error" in response.lower():
            return "ERROR"
        if ((time.time()-start_time) > 10):
            return "TIMEOUT"

    return "OK"

In [7]:
AT(device, "AT+SEND=eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee")

'OK'

In [30]:
device.close()

In [22]:
response = device.read_all().decode()
response = response

In [23]:
print(response)


OK



In [7]:
generate_packets(20, 50)

(['SOT',
  'TIME=1687858314',
  'N=20',
  '58794212204733981822755614892043880546219848844134',
  '46609072618046046503058708483669819573823467641805',
  '32156215713129421322961068321141297825260145971305',
  '80692009792339714662940070365763878195161541255529',
  '52132999191248321528891275547947969192065278213606',
  '54567904453354802691207119872740443306199904053723',
  '69428612785598401173295661955904464692339432974171',
  '09138828256554621645819042305431436624780240048714',
  '80416483252948873289022615006805155240853673629269',
  '00866059036345392666733643638218979859425854016780',
  '89962196945617718284829088711013515514795161351159',
  '15270129955764006093465982087683348804704129387344',
  '08370822917288838704446611764352014060218429819714',
  '99395507144807057599815525651587366383146335694240',
  '26780442640489062223684111354813162620359888890794',
  '86948932663008928401683961398988774027228095149591',
  '84415234726640733954836007854537725637015766443291',
  '27275

In [24]:
import pandas as pd
import os

In [25]:
l = [0,1,2,4,5]

df = pd.DataFrame()
for item in l:
    df[f"DF{item}"] = pd.read_csv(f"./data/AOI measurements/30_30/AoI_{item}.csv")["AoI"]
df.index.name="Time"
df.index = df.index * 30
df

,DF0,DF1,DF2,DF4,DF5
Time,,,,,
0,184,143,124,103,104
30,214,173,154,133,134
60,244,203,184,163,164
90,173,131,113,103,104
120,203,161,143,133,134
150,233,191,173,163,164
180,163,121,104,193,104
210,193,151,134,104,134
240,150,110,164,134,164


In [26]:
import numpy as np

In [30]:
avgs, stds = np.mean(df.to_numpy(), axis=0), np.std(df.to_numpy(),axis=0)
avgs, stds

pd.DataFrame({"mean": avgs, "std": stds}, index=df.columns)

,mean,std
DF0,160.600000,37.825565
DF1,139.533333,28.015155
DF2,136.200000,25.380307
DF4,135.133333,27.777369
DF5,133.800000,24.498163


In [28]:
from plotly import express as px

In [29]:
fig = px.line(df, x=df.index, y=df.columns)
fig.show()